In [91]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

from pycaret.regression import *

import joblib

In [94]:
df = pd.read_csv('survey_results_public.csv.gz', compression='gzip')
df.sample(3)

,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,...,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech,TrueFalse_1,TrueFalse_2,TrueFalse_3,SurveyLength,SurveyEase,ConvertedCompYearly
51851,51852,I am a developer by profession,"Employed, full-time",Full in-person,Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Other online resources ...,Technical documentation;Blogs;Written Tutorial...,Coursera;Codecademy,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,NaN
53542,53543,"I am not primarily a developer, but I write co...","Employed, full-time",Fully remote,Hobby;Contribute to open-source projects;Boots...,"Associate degree (A.A., A.S., etc.)",Books / Physical media;On the job training,NaN,NaN,41,...,15-30 minutes a day,30-60 minutes a day,NaN,Innersource initiative;DevOps function;Microse...,Yes,Yes,No,Appropriate in length,Easy,230000.0
12924,12925,I am a developer by profession,"Employed, full-time",Full in-person,Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Friend or family member;Other online resources...,Stack Overflow;Video-based Online Courses;How-...,NaN,12,...,30-60 minutes a day,15-30 minutes a day,Somewhat long,DevOps function,Yes,No,Yes,Too long,Neither easy nor difficult,NaN


In [4]:
df.columns.unique()

Index(['ResponseId', 'MainBranch', 'Employment', 'RemoteWork',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       'LearnCodeCoursesCert', 'YearsCode', 'YearsCodePro', 'DevType',
       'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'Country', 'Currency',
       'CompTotal', 'CompFreq', 'LanguageHaveWorkedWith',
       'LanguageWantToWorkWith', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'PlatformHaveWorkedWith',
       'PlatformWantToWorkWith', 'WebframeHaveWorkedWith',
       'WebframeWantToWorkWith', 'MiscTechHaveWorkedWith',
       'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith',
       'ToolsTechWantToWorkWith', 'NEWCollabToolsHaveWorkedWith',
       'NEWCollabToolsWantToWorkWith', 'OpSysProfessional use',
       'OpSysPersonal use', 'VersionControlSystem', 'VCInteraction',
       'VCHostingPersonal use', 'VCHostingProfessional use',
       'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',
       'OfficeStackSyncHaveWork

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73268 entries, 0 to 73267
Data columns (total 79 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ResponseId                      73268 non-null  int64  
 1   MainBranch                      73268 non-null  object 
 2   Employment                      71709 non-null  object 
 3   RemoteWork                      58958 non-null  object 
 4   CodingActivities                58899 non-null  object 
 5   EdLevel                         71571 non-null  object 
 6   LearnCode                       71580 non-null  object 
 7   LearnCodeOnline                 50685 non-null  object 
 8   LearnCodeCoursesCert            29389 non-null  object 
 9   YearsCode                       71331 non-null  object 
 10  YearsCodePro                    51833 non-null  object 
 11  DevType                         61302 non-null  object 
 12  OrgSize                         

In [6]:
df['Country'].value_counts()[:3]

Country
United States of America    13543
India                        6639
Germany                      5395
Name: count, dtype: int64

In [7]:
def plot_line_chart(df, columns, line=''):
    if line == '':
        line = df[columns].value_counts().keys()[:20]
    data = df[columns].value_counts()[:20]
    fig = px.line(x=line, y=data)
    fig.show()

def plot_bar_chart(df, columns, line=''):
    if line == '':
        line = df[columns].value_counts().keys()[:20]
    data = df[columns].value_counts()[:20]
    fig = px.bar(x=line, y=data)
    fig.show()

def plot_pie_chart(df, columns, line=''):
    if line == '':
        line = df[columns].value_counts().keys()[:20]
    data = df[columns].value_counts()[:20]
    fig = px.pie(names=line, values=data)
    fig.show()

In [8]:
plot_pie_chart(df, columns='Country',)

In [9]:
plot_bar_chart(df, columns='Country',)

In [10]:
plot_bar_chart(df, columns='YearsCodePro')

In [11]:
df = df[['Country','EdLevel','YearsCodePro','ConvertedCompYearly']]
df = df.rename({'ConvertedCompYearly': 'Salary'}, axis=1)
df.sample(5)

,Country,EdLevel,YearsCodePro,Salary
19540,Bulgaria,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",2,NaN
15356,Turkey,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",13,37584.0
35093,United Kingdom of Great Britain and Northern I...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",2,54025.0
30613,United States of America,Some college/university study without earning ...,1,70000.0
20798,United Kingdom of Great Britain and Northern I...,"Associate degree (A.A., A.S., etc.)",NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73268 entries, 0 to 73267
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country       71771 non-null  object 
 1   EdLevel       71571 non-null  object 
 2   YearsCodePro  51833 non-null  object 
 3   Salary        38071 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.2+ MB


In [13]:
df = df[df['Salary'].notnull()]
df.head()

,Country,EdLevel,YearsCodePro,Salary
2,United Kingdom of Great Britain and Northern I...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,40205.0
3,Israel,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",17,215232.0
8,Netherlands,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",6,49056.0
10,United Kingdom of Great Britain and Northern I...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",2,60307.0
11,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,194400.0


In [14]:
df = df.dropna()

In [15]:
df.isnull().sum()

Country         0
EdLevel         0
YearsCodePro    0
Salary          0
dtype: int64

In [16]:
df['Country'].value_counts()

Country
United States of America                                8684
Germany                                                 2901
United Kingdom of Great Britain and Northern Ireland    2649
India                                                   2155
Canada                                                  1478
                                                        ... 
Fiji                                                       1
Palau                                                      1
Monaco                                                     1
Saint Lucia                                                1
Seychelles                                                 1
Name: count, Length: 159, dtype: int64

In [17]:
def unique_col_val(df):
    for col in df:
        if df[col].dtypes == 'object':
            print(f'{col}: {df[col].unique()}')

In [18]:
unique_col_val(df)

Country: ['United Kingdom of Great Britain and Northern Ireland' 'Israel'
 'Netherlands' 'United States of America' 'Czech Republic' 'Austria'
 'Italy' 'Canada' 'Germany' 'Ireland' 'Poland' 'Madagascar' 'Norway'
 'Taiwan' 'France' 'Brazil' 'Uruguay' 'Sweden' 'Spain' 'Turkey' 'Romania'
 'Singapore' 'India' 'Belgium' 'Bulgaria' 'Greece' 'Portugal'
 'Russian Federation' 'Saudi Arabia' 'Mexico' 'Kenya' 'Switzerland'
 'Latvia' 'South Africa' 'Thailand' 'China' 'Montenegro' 'Finland'
 'Slovakia' 'Japan' 'Denmark' 'Australia' 'Viet Nam' 'Argentina' 'Hungary'
 'Tunisia' 'Bangladesh' 'Ukraine' 'Maldives' 'Hong Kong (S.A.R.)' 'Egypt'
 'Serbia' 'Pakistan' 'Nepal' 'Croatia' 'Indonesia'
 'Bosnia and Herzegovina' 'Armenia' 'Lithuania'
 'Iran, Islamic Republic of...' 'Belarus' 'Costa Rica' 'Mauritius'
 'Estonia' 'Kazakhstan' 'Morocco' 'Philippines' 'Chile' 'New Zealand'
 'Slovenia' 'Ecuador' 'Cyprus' 'Peru' 'Colombia' 'Afghanistan' 'Nicaragua'
 'Andorra' 'Republic of Korea' 'Lebanon' 'South Korea' 'M

In [19]:
def shorten_categories(categories, cutoff):
    categoriecal_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categoriecal_map[categories.index[i]] = categories.index[i]
        else:
            categoriecal_map[categories.index[i]] = 'Other'
            
    return categoriecal_map

In [20]:
country_map = shorten_categories(df.Country.value_counts(), 199)
df['Country'] = df['Country'].map(country_map)
df.Country.value_counts()

Country
United States of America                                8684
Other                                                   4356
Germany                                                 2901
United Kingdom of Great Britain and Northern Ireland    2649
India                                                   2155
Canada                                                  1478
France                                                  1372
Brazil                                                  1279
Spain                                                    993
Poland                                                   973
Netherlands                                              897
Australia                                                830
Italy                                                    807
Sweden                                                   664
Russian Federation                                       533
Switzerland                                              520
Turkey          

In [21]:
df = df[df['Salary'] <= 200000]
df = df[df['Salary'] >= 10000]
df = df[df['Country'] != 'Other']

In [22]:
df.head()

,Country,EdLevel,YearsCodePro,Salary
2,United Kingdom of Great Britain and Northern I...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5,40205.0
8,Netherlands,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",6,49056.0
10,United Kingdom of Great Britain and Northern I...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",2,60307.0
11,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",10,194400.0
12,United States of America,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",5,65000.0


In [23]:
df['YearsCodePro'].unique()

array(['5', '6', '2', '10', '4', '22', '20', '9', '14', '21', '15', '3',
       '25', '7', '8', '12', '19', '1', '17', '24', '11', '23',
       'Less than 1 year', '18', '40', '37', '46', '13', '32', '31', '39',
       '27', '28', '16', '30', '34', '35', '26', '42', '38', '36', '43',
       '41', '44', '29', '33', '45', '48', '50', '47',
       'More than 50 years', '49'], dtype=object)

In [24]:
def clean_exprience(x):
    if x == 'More than 50 years':
        return 50
    if x == 'Less than 1 year':
        return 0.5
    return float(x)

In [25]:
df['YearsCodePro'] = df['YearsCodePro'].apply(clean_exprience)

In [26]:
df.YearsCodePro.value_counts()

YearsCodePro
5.0     2362
3.0     2187
4.0     2122
10.0    1909
6.0     1904
2.0     1836
7.0     1676
8.0     1586
1.0     1151
15.0    1018
12.0    1006
9.0      973
0.5      878
11.0     826
20.0     728
14.0     627
13.0     563
22.0     516
16.0     503
25.0     460
18.0     440
17.0     430
30.0     261
24.0     252
23.0     248
21.0     238
19.0     203
26.0     161
27.0     159
35.0     142
28.0     129
32.0     116
40.0      68
29.0      61
36.0      59
34.0      59
37.0      56
31.0      56
33.0      54
38.0      51
42.0      29
41.0      22
39.0      20
43.0      19
45.0      16
50.0      16
44.0      12
46.0       8
48.0       4
47.0       4
49.0       4
Name: count, dtype: int64

In [27]:
df['EdLevel'].unique()

array(['Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Something else', 'Associate degree (A.A., A.S., etc.)',
       'Some college/university study without earning a degree',
       'Professional degree (JD, MD, etc.)',
       'Other doctoral degree (Ph.D., Ed.D., etc.)',
       'Primary/elementary school'], dtype=object)

In [28]:
def clean_education(x):
    if 'Master’s degree' in x:
        return 'Master’s degree'
    if 'Bachelor’s degree' in x:
        return 'Bachelor’s degree'
    if 'Professional degree' in x or 'Other doctoral' in x:
        return 'Less than a Bachelors'

In [29]:
df['EdLevel'] = df["EdLevel"].apply(clean_education)

In [30]:
df['EdLevel'].unique()

array(['Master’s degree', 'Bachelor’s degree', None,
       'Less than a Bachelors'], dtype=object)

In [31]:
df.sample(10)

,Country,EdLevel,YearsCodePro,Salary
51746,Pakistan,Master’s degree,12.0,41892.0
20280,Portugal,Bachelor’s degree,10.0,53322.0
41621,United States of America,None,15.0,155000.0
11793,United Kingdom of Great Britain and Northern I...,None,8.0,94230.0
3418,United States of America,Bachelor’s degree,1.0,140000.0
53624,United States of America,Bachelor’s degree,6.0,130000.0
5859,Italy,None,11.0,26772.0
70189,Ireland,Master’s degree,3.0,117308.0
20474,United States of America,Bachelor’s degree,6.0,110000.0
36698,United Kingdom of Great Britain and Northern I...,Bachelor’s degree,3.0,138203.0


In [32]:
X = df.drop('Salary', axis=1)
y = df['Salary']

In [59]:
numeric_pipeline = Pipeline(steps= [('scaler', StandardScaler())])

categorical_pipeline = Pipeline(steps= [('encoder', OneHotEncoder(handle_unknown='ignore'))])

transformer = ColumnTransformer(transformers=[('num', numeric_pipeline, ['YearsCodePro']), ('cat', categorical_pipeline, ['Country', 'EdLevel'])
])

mlpipe_xgb = Pipeline(steps= [('transformer', transformer), ('xgboost', XGBRegressor(random_state=15))])
mlpipe_gbr = Pipeline(steps= [('transformer', transformer), ('gbr', GradientBoostingRegressor(random_state=15))])
mlpipe_knr = Pipeline(steps= [('transformer', transformer), ('knr', KNeighborsRegressor())])
mlpipe_ann = Pipeline(steps= [('transformer', transformer), ('ann', MLPRegressor(random_state=15))])
mlpipe_lnr = Pipeline(steps= [('transformer', transformer), ('lnr', LinearRegression())])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [36]:
setup(data=X, target=y, session_id=15)

,Description,Value
0,Session id,15
1,Target,Salary
2,Target type,Regression
3,Original data shape,"(28228, 4)"
4,Transformed data shape,"(28228, 6)"
5,Transformed train set shape,"(19759, 6)"
6,Transformed test set shape,"(8469, 6)"
7,Numeric features,1
8,Categorical features,2
9,Rows with missing values,21.7%


In [37]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,22970.8016,971273425.0235,31161.0334,0.5590,0.4432,0.4018,0.0690
lightgbm,Light Gradient Boosting Machine,22990.1043,976573016.0309,31245.6499,0.5566,0.4430,0.4002,0.0600
xgboost,Extreme Gradient Boosting,23296.4057,1006310073.6000,31717.5811,0.5430,0.4495,0.4040,0.0300
lasso,Lasso Regression,24158.3502,1047456992.0662,32359.3007,0.5246,0.4625,0.4275,0.1450
ridge,Ridge Regression,24158.3412,1047456887.1369,32359.2989,0.5246,0.4625,0.4275,0.0220
lar,Least Angle Regression,24158.3428,1047456983.3164,32359.3005,0.5246,0.4625,0.4275,0.0160
llar,Lasso Least Angle Regression,24158.3501,1047456989.8190,32359.3006,0.5246,0.4625,0.4275,0.0160
br,Bayesian Ridge,24158.3125,1047455593.5368,32359.2772,0.5246,0.4625,0.4276,0.0170
lr,Linear Regression,24158.3428,1047456983.3164,32359.3005,0.5246,0.4625,0.4275,0.2890
en,Elastic Net,24203.0233,1050133154.5871,32400.6664,0.5234,0.4637,0.4303,0.0160


GradientBoostingRegressor(random_state=15)

In [38]:
X_train.head()

,Country,EdLevel,YearsCodePro
57887,Australia,None,8.0
10513,Belgium,Bachelor’s degree,3.0
61357,Canada,None,3.0
35578,Switzerland,Bachelor’s degree,11.0
40620,United States of America,None,7.0


In [66]:
def parametr_finder(model, parameters):

    grid = GridSearchCV(model,
                        param_grid= parameters,
                        refit = True,
                        cv= KFold(shuffle=True, random_state=1),
                        n_jobs= -1)
    grid_fit = grid.fit(X_train, y_train)
    y_pred_train = grid_fit.predict(X_train)
    y_pred = grid_fit.predict(X_test)

    train_score = grid_fit.score(X_train, y_train)
    test_score = grid_fit.score(X_test, y_test)
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

    model_name = str(model).split('(')[0]

    print(f"The best parameters for {model_name} model is: {grid_fit.best_params_}")
    print("--" * 10)
    print(f"(R2 score) in the training set is {train_score:0.2%} for {model_name} model.")
    print(f"(R2 score) in the testing set is {test_score:0.2%} for {model_name} model.")
    print(f"RMSE is {RMSE:,} for {model_name} model.")
    print("--" * 10)

    return train_score, test_score, RMSE

In [67]:
xgboost_params = {
    'xgboost__max_depth': [4,5,6],
    'xgboost__n_estimators': [256,350],
    'xgboost__learning_rate': [0.1, 0.125]
    }

xgboost_train_score, xgboost_test_score, xgboost_RMSE = parametr_finder(mlpipe_xgb, xgboost_params)

The best parameters for Pipeline model is: {'xgboost__learning_rate': 0.1, 'xgboost__max_depth': 4, 'xgboost__n_estimators': 256}
--------------------
(R2 score) in the training set is 58.44% for Pipeline model.
(R2 score) in the testing set is 56.94% for Pipeline model.
RMSE is 30,403.911227487748 for Pipeline model.
--------------------


In [72]:
knr_params = {
            'knr__n_neighbors': [5, 10, 20, 30],
            'knr__weights': ['uniform', 'distance']
            }

knr_train_score, knr_test_score, knr_RMSE = parametr_finder(mlpipe_knr, knr_params)

The best parameters for Pipeline model is: {'knr__n_neighbors': 30, 'knr__weights': 'uniform'}
--------------------
(R2 score) in the training set is 56.66% for Pipeline model.
(R2 score) in the testing set is 54.72% for Pipeline model.
RMSE is 31,177.27400445044 for Pipeline model.
--------------------


In [73]:
gbr_params = {
    'gbr__n_estimators': [64,128,256],
    'gbr__learning_rate': [0.1, 0.125],
    'gbr__max_depth': [4,5,6]
    }

gbr_train_score, gbr_test_score, gbr_RMSE = parametr_finder(mlpipe_gbr, gbr_params)

The best parameters for Pipeline model is: {'gbr__learning_rate': 0.125, 'gbr__max_depth': 4, 'gbr__n_estimators': 128}
--------------------
(R2 score) in the training set is 57.98% for Pipeline model.
(R2 score) in the testing set is 56.91% for Pipeline model.
RMSE is 30,415.03403729366 for Pipeline model.
--------------------


In [74]:
ann_params = {
    'ann__hidden_layer_sizes': [64,128,256,512],
    'ann__alpha': [0.1,0.01,0.001],
    'ann__max_iter': [128,256]
}

ann_train_score, ann_test_score, ann_RMSE = parametr_finder(mlpipe_ann, ann_params)

The best parameters for Pipeline model is: {'ann__alpha': 0.1, 'ann__hidden_layer_sizes': 512, 'ann__max_iter': 256}
--------------------
(R2 score) in the training set is 53.98% for Pipeline model.
(R2 score) in the testing set is 54.57% for Pipeline model.
RMSE is 31,230.71252172272 for Pipeline model.
--------------------


In [75]:
lnr_train_score, lnr_test_score, lnr_RMSE = parametr_finder(mlpipe_lnr, {})

The best parameters for Pipeline model is: {}
--------------------
(R2 score) in the training set is 53.05% for Pipeline model.
(R2 score) in the testing set is 53.59% for Pipeline model.
RMSE is 31,562.74747446091 for Pipeline model.
--------------------


In [76]:
mlpipe_final_xgb = Pipeline(steps= [('transformer', transformer), ('xgboost', XGBRegressor(learning_rate= 0.1, xgboost__max_depth= 4, n_estimators= 256, random_state=15))])


In [78]:
mlpipe_final_xgb.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['YearsCodePro']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Country', 'EdLevel'])])),
                ('xgboost',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              c...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=256, n_jobs=None,
                              num_parallel_tree=None, random_state=15, ...))])

In [79]:
mlpipe_final_xgb.predict(X_test)

array([ 68486.55 ,  68299.65 , 138433.28 , ...,  54984.836, 133277.72 ,
        91804.72 ], dtype=float32)

In [82]:
model_path = 'xgb.joblib'
joblib.dump(mlpipe_final_xgb, model_path)

['xgb.joblib']

In [83]:
model = joblib.load('xgb.joblib')

In [84]:
model.predict(X_test)

array([ 68486.55 ,  68299.65 , 138433.28 , ...,  54984.836, 133277.72 ,
        91804.72 ], dtype=float32)